# Lab 12-5 sequence to sequence (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

In [1]:
pip install tensorflow==1.15

  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
import pandas as pd
import re
#요약
from gensim.summarization.summarizer import summarize




1.15.0


In [2]:

#path = '/content/drive/My Drive/Colab Notebooks/moviedata2.json'
df = pd.read_json('moviedata2.json')


# 전처리

In [6]:
#df = movie_dataframe

df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. 0-9]", '', x))
df.movie_summary = df.movie_summary.apply(lambda x: x.replace('\\n',''))

#df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x).lower())
#df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))

df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n','').replace('\\xa0',''))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. 0-9]', '', x))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))

#태그라인 없는 데이터 row 삭제
index = df[df['movie_tagline']==''].index
df.drop(index, inplace = True)

#summary 없는 데이터 row 삭제
summary_index = df[df['movie_summary']==''].index
df.drop(summary_index, inplace = True)

index = df[df['movie_summary'].str.contains('  ')].index
df.drop(index , inplace = True)

index = df[df['movie_summary']==''].index
df.drop(index, inplace=True)
df = df.reset_index()
df.drop(df.index[-172:], inplace = True)





In [7]:
df

,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,0,"[Adventure, Drama, Horror, Thriller]",47 Meters Down: Uncaged,www.imdb.com/title/tt7329656/mediaviewer/rm242...,four teen girls diving in a ruined underwater ...,"[The film starts at a girls school in Yucatan,...",the next chapter
1,2,"[Horror, Sci-Fi]",Alien,www.imdb.com/title/tt0078748/mediaviewer/rm299...,after a space merchant vessel perceives an unk...,[The opening credits appear in front of a larg...,its alien the 8th passenger.
2,4,"[Adventure, Drama, Fantasy]",The Lord of the Rings: The Fellowship of the Ring,www.imdb.com/title/tt0120737/mediaviewer/rm359...,a meek hobbit from the shire and eight compani...,"[The prologue, spoken by Galadriel, shows the ...",all we have to decide is what to do with the ...
3,5,"[Crime, Drama, Thriller]",The Mule,www.imdb.com/title/tt7959026/mediaviewer/rm320...,a 90yearold horticulturist and korean war vete...,[This movie is based on a true story of a Worl...,nobody runs forever
4,6,"[Adventure, Family, Fantasy]",Harry Potter and the Sorcerer's Stone,www.imdb.com/title/tt0241527/mediaviewer/rm683...,an orphaned boy enrolls in a school of wizardr...,"[Lord Voldemort, an evil and powerful dark wiz...",the magic begins november 16th.
5,7,"[Comedy, Family, Fantasy]",Hocus Pocus,www.imdb.com/title/tt0107120/mediaviewer/rm354...,a curious youngster moves to salem where he st...,"[Hocus Pocus begins in a small town in 1693, w...",its just a bunch of hocus pocus
6,18,"[Horror, Mystery, Thriller]",Friday the 13th,www.imdb.com/title/tt0758746/mediaviewer/rm380...,a group of young adults discover a boarded up ...,"[\n , \n ]",you know his name. you know the story. on fri...
7,21,"[Action, Crime, Horror]",From Dusk Till Dawn,www.imdb.com/title/tt0116367/mediaviewer/rm147...,two criminals and their hostages unknowingly s...,[Texas sheriff Edgar McGraw pulls up to an iso...,vampires. no interviews.
8,23,"[Drama, Western]",Django Unchained,www.imdb.com/title/tt1853728/mediaviewer/rm958...,with the help of a german bounty hunter a free...,"[In 1858, Django (, ), a slave, is chained to ...",the d is silent. payback wont be.
9,24,"[Drama, Sport]",Overcomer,www.imdb.com/title/tt8186318/mediaviewer/rm428...,a highschool basketball coach volunteers to co...,[The opening scene takes us into the gym of Br...,what do you allow to define you


In [616]:
df

,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,0,"[Adventure, Drama, Horror, Thriller]",47 Meters Down: Uncaged,www.imdb.com/title/tt7329656/mediaviewer/rm242...,four teen girls diving in a ruined underwater ...,"[The film starts at a girls school in Yucatan,...",the next chapter
1,2,"[Horror, Sci-Fi]",Alien,www.imdb.com/title/tt0078748/mediaviewer/rm299...,after a space merchant vessel perceives an unk...,[The opening credits appear in front of a larg...,its alien the 8th passenger.
2,4,"[Adventure, Drama, Fantasy]",The Lord of the Rings: The Fellowship of the Ring,www.imdb.com/title/tt0120737/mediaviewer/rm359...,a meek hobbit from the shire and eight compani...,"[The prologue, spoken by Galadriel, shows the ...",all we have to decide is what to do with the ...
3,5,"[Crime, Drama, Thriller]",The Mule,www.imdb.com/title/tt7959026/mediaviewer/rm320...,a 90yearold horticulturist and korean war vete...,[This movie is based on a true story of a Worl...,nobody runs forever
4,6,"[Adventure, Family, Fantasy]",Harry Potter and the Sorcerer's Stone,www.imdb.com/title/tt0241527/mediaviewer/rm683...,an orphaned boy enrolls in a school of wizardr...,"[Lord Voldemort, an evil and powerful dark wiz...",the magic begins november 16th.
5,7,"[Comedy, Family, Fantasy]",Hocus Pocus,www.imdb.com/title/tt0107120/mediaviewer/rm354...,a curious youngster moves to salem where he st...,"[Hocus Pocus begins in a small town in 1693, w...",its just a bunch of hocus pocus
6,18,"[Horror, Mystery, Thriller]",Friday the 13th,www.imdb.com/title/tt0758746/mediaviewer/rm380...,a group of young adults discover a boarded up ...,"[\n , \n ]",you know his name. you know the story. on fri...
7,21,"[Action, Crime, Horror]",From Dusk Till Dawn,www.imdb.com/title/tt0116367/mediaviewer/rm147...,two criminals and their hostages unknowingly s...,[Texas sheriff Edgar McGraw pulls up to an iso...,vampires. no interviews.
8,23,"[Drama, Western]",Django Unchained,www.imdb.com/title/tt1853728/mediaviewer/rm958...,with the help of a german bounty hunter a free...,"[In 1858, Django (, ), a slave, is chained to ...",the d is silent. payback wont be.
9,24,"[Drama, Sport]",Overcomer,www.imdb.com/title/tt8186318/mediaviewer/rm428...,a highschool basketball coach volunteers to co...,[The opening scene takes us into the gym of Br...,what do you allow to define you


In [617]:
#summary 문장 수가 너무 많을 경우 summarize를 통해 압축
# for idx, summary in enumerate(df['movie_summary']):
#     x = summary.split('. ')
#     #print(x)
#     if len(x) <= 2:
#         df['movie_summary'][idx] = summary
#     else:
#         df['movie_summary'][idx] = summarize(summary, word_count=50,ratio = 0.3)


index = df[df['movie_summary']==''].index
df.drop(index, inplace=True)
df = df.reset_index()
df.drop(df.index[-712:], inplace = True)




In [8]:

tl = list(df["movie_tagline"])
tl = ["".join([ j for j in i if j != "\n"]).strip() for i in tl]

summ = list(df['movie_summary'])

df.movie_tagline = tl
df.movie_summary = summ

sources = [i.split() for i in df['movie_summary']]
targets = [i.split() for i in df['movie_tagline']]


In [9]:
df.tail()

,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
9343,41971,"[Comedy, Drama]",Dark Arc,www.imdb.com/title/tt0429980/mediaviewer/rm101...,a mysterious comedy about love lust art and th...,"[\n , \n ]",for arrogant bastards only...
9344,41974,[Drama],The Last Supper,www.imdb.com/title/tt0110310/mediaviewer/rm190...,chris is a dancer dying of aids. he has chosen...,"[\n , \n ]",we are all gathered together here because you ...
9345,41983,"[Comedy, Drama]",The Sandpiper,www.imdb.com/title/tt0448106/mediaviewer/rm910...,ethan and terry are best summer friends who bo...,"[\n , \n ]",three is a magic number.
9346,41998,"[Animation, Comedy, Sci-Fi]",The X Mas Files,www.imdb.com/title/tt4994884/mediaviewer/rm495...,ever wondered how santa flies his sleigh two w...,"[\n , \n ]",this christmas... discover the truth.
9347,42011,"[Drama, Romance]",Evidence,www.imdb.com/title/tt0019859/mediaviewer/rm324...,evidence is a 1929 precode crime drama film pr...,"[\n , \n ]",a perfect talking picture with a perfect talki...


In [620]:
df.to_json('moviedata_preprocessed.json')

# 단어 나누기

In [ ]:

# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}



In [ ]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}




In [ ]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        
        #s_input = [[x for x in each_input if x] for each_input in s_input]
        #pprint(s_input)
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        
        #t_input = [[x for x in each_input if x] for each_input in t_input]
        #pprint(t_input)
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
     
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [554]:
sum_ = 0
for i in df.movie_summary:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(df) #summary당 평균 122.84 단어
# preprocessing for source
#단어 수는 120개로 제한
s_max_len = int(average_no) + 30
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
#print(s_len, s_input)

In [555]:
sum_ = 0
for i in df.movie_tagline:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(df) #각 tagline별 단어 수는 9.6개

# preprocessing for target
#단어 수 10개로 제한
t_max_len = int(average_no)
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

[5, 7, 18, 5, 7, 9, 17, 5, 9, 9, 9, 6, 5, 5, 25, 8, 8, 8, 4, 8, 5, 8, 26, 17, 7, 6, 13, 11, 47, 6, 12, 13, 6, 10, 15, 8, 6, 17, 9, 9, 75, 35, 12, 8, 4, 10, 12, 8, 13, 8, 4, 12, 22, 4, 7, 6, 4, 10, 5, 23, 8, 8, 5, 7, 10, 10, 7, 12, 7, 5, 7, 6, 23, 7, 6, 5, 12, 6, 10, 14, 19, 4, 12, 7, 6, 15, 8, 8, 6, 9, 37, 15, 6, 5, 13, 8, 11, 7, 13, 10, 7, 5, 9, 6, 7, 9, 6, 16, 10, 14, 9, 10, 16, 11, 8, 6, 10, 6, 8, 8, 21, 11, 10, 23, 6, 6, 5, 13, 10, 6, 10, 12, 5, 7, 8, 5, 7, 5, 12, 8, 9, 10, 8, 8, 6, 4, 9, 4, 6, 11, 5, 14, 11, 5, 11, 12, 13, 6, 7, 29, 10, 8, 13, 10, 10, 7, 7, 5, 11, 10, 6, 7, 26, 7, 9, 7, 8, 8, 9, 11, 6, 7, 11, 6, 8, 6, 14, 5, 9, 16, 11, 18, 6, 45, 9, 5, 8, 7, 9, 23, 9, 9, 13, 16, 8, 9, 9, 17, 4, 7, 10, 14, 5, 6, 7, 12, 8, 9, 5, 17, 5, 14, 26, 10, 11, 5, 8, 8, 6, 11, 7, 7, 13, 6, 13, 15, 6, 7, 26, 9, 12, 10, 12, 16, 9, 8, 11, 11, 6, 15, 7, 8, 12, 8, 5, 10, 8, 11, 10, 9, 9, 6, 9, 16, 10, 8, 6, 16, 12, 10, 6, 12, 8, 9, 6, 8, 20, 15, 14, 9, 7, 19, 12, 9, 6, 6, 14, 8, 13, 10, 11, 9, 15,

# hyper-param

In [556]:
# hyper-parameters
epochs = 100
batch_size = 4
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 32

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [557]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [558]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [559]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
                
    def call(self, x, hidden, enc_output):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [560]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)
print(checkpoint_prefix)
# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)
print(summary_writer)

./data_out/training_checkpoints\ckpt


In [85]:
EPOCHS = 1

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    #print(hidden.shape)
    total_loss = 0
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        #print(s_input, s_input.shape)
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)

            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                
                predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))


#     if epoch % 10 == 0:
#         #save model every 10 epoch
#         print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
#                                             total_loss / n_batch,
#                                             batch_loss.numpy()))
#         optimizer.minimize(variables)
#         checkpoint.save(file_prefix = checkpoint_prefix)


#save model every 10 epoch
print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                    total_loss / n_batch,
                                    batch_loss.numpy()))
optimizer.minimize(variables)
checkpoint.save(file_prefix = checkpoint_prefix)

Epoch 0 Loss 182.4760 Batch Loss 4.0433


RuntimeError: `loss` passed to Optimizer.compute_gradients should be a function when eager execution is enabled.

In [545]:
# #최근 학습된 모델을 불러오고 이어서 EPOCHS로 돌아감
# saver = tf.train.Saver()
# with tf.Session() as sess:
    
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [591]:
#sentence = 'I feel hungry'
sentence = 'donghoon teens are eat minsoo'
print(sentence)
print(df.movie_name[7])
print(df.movie_tagline[7])
#sentence = 'desperado'
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

donghoon teens are eat minsoo
From Dusk Till Dawn
vampires. no interviews.


In [592]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
        
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
        
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
   
    for t in range(max_length_targ):
        

        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '
        
        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)    
            
    return result, sentence
    
result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(sentence)
print(result)

[37494, 2510, 12072]
donghoon teens are eat minsoo
until prodigal energy haunted tell. pilot...grand redemption. timing spoke balaguer. jericho heart 


In [112]:
#len(df.movie_summary[39].split('. '))

In [593]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    print(inputs)
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    #print(dec_input)
    xList = []
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        #print("*"*100)
        #print(predictions)
        #print("*"*100)
        #print(tf.argmax(predictions[0]))
        print(predictions)
        x = tf.nn.top_k(predictions[0], k=3, sorted=True, name=None)
        for i in range(3):
            predicted_id = x[1][i].numpy()
            #print(predicted_id)

            result += idx2target[predicted_id] + ' '
            #print("result:", result)
            #print(idx2target[predicted_id])
            #if idx2target.get(predicted_id) == '<eos>':
            #             return result, sentence

          #result=""

        # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)  
        xList.append([result])
        result = ''
        
    return xList

# result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
tList=prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

tf.Tensor([[37494  2510 12072     0     0     0     0     0     0     0]], shape=(1, 10), dtype=int32)
tf.Tensor(
[[ 0.00192981  0.00387653 -0.0001338  ...  0.00084078  0.00136333
  -0.00058426]], shape=(1, 7883), dtype=float32)
tf.Tensor(
[[3.0533713e-04 3.6703222e-03 7.5694861e-06 ... 2.1257490e-04
  9.7969046e-04 6.0740899e-04]], shape=(1, 7883), dtype=float32)
tf.Tensor(
[[-0.00073569  0.00345075  0.00034872 ... -0.00028084  0.00050289
   0.00119432]], shape=(1, 7883), dtype=float32)
tf.Tensor(
[[-0.00100886  0.00106799  0.00237389 ...  0.00022285 -0.00064127
   0.00126915]], shape=(1, 7883), dtype=float32)
tf.Tensor(
[[-1.8556989e-03  1.7531029e-03  7.8196794e-04 ... -2.2061402e-03
   6.1286998e-04 -2.2338849e-05]], shape=(1, 7883), dtype=float32)
tf.Tensor(
[[-1.1637511e-03 -8.2672969e-04  5.2065455e-04 ... -5.8668706e-04
  -4.1378371e-04  3.7095942e-05]], shape=(1, 7883), dtype=float32)
tf.Tensor(
[[-0.00093286 -0.00019236  0.00069534 ... -0.00250729 -0.00149696
  -0.00030172]],